In [82]:
##complex data와 똑같이 40ms로 parsing했음 데이터 4배라고 생각하고 뻥튀기시켜봄 정확도는 10ms와 비슷하게 나옴

import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
from sklearn import svm
from datetime import datetime
import glob
import pickle
import re
from sklearn import preprocessing
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import GridSearchCV
from scipy import signal

#filename queue
filename_queue=glob.glob('./label/label[0-9]*.csv')
#filename_queue=glob.glob('./label/label[6].csv')


In [94]:
"""training_set2=list()
training_label2=list()
test_set2=list()
test_label2=list()
traininglabel_count2=np.zeros(16)
testlabel_count2=np.zeros(16)
"""
training_set=list()

test_set=list()
training_label=list()
test_label=list()
index=[2,3,4,11,14,15]
label_count=np.zeros(16)
traininglabel_count=np.zeros(16)
testlabel_count=np.zeros(16)
ldata=list()

for filename in filename_queue:
    with open(filename,newline='') as data:
        
        data_set=list()
        data_set2=list()
        data_label=list()
        reader=csv.reader(data)
        label=re.findall('\d+',filename)
        label=int(label[0])
        label_count[label]+=1
        if label in index:
            continue
        if label not in index:
            print('label : ', label)
            for row in reader:
                row[0:]=[float(i) for i in row[0:]]
                X_acc=list(row[i] for i in range(150))
                Y_acc=list(row[i+150] for i in range(150))
                Z_acc=list(row[i+300] for i in range(150))
                X_gyro=list(row[i+450] for i in range(150))
                Y_gyro=list(row[i+600] for i in range(150))
                Z_gyro=list(row[i+750] for i in range(150))

                window=np.array([X_acc[0:25],Y_acc[0:25],Z_acc[0:25],X_gyro[0:25],Y_gyro[0:25],Z_gyro[0:25],
                                  X_acc[25:50],Y_acc[25:50],Z_acc[25:50],X_gyro[25:50],Y_gyro[25:50],Z_gyro[25:50],
                                  X_acc[50:75],Y_acc[50:75],Z_acc[50:75],X_gyro[50:75],Y_gyro[50:75],Z_gyro[50:75],
                                  X_acc[75:100],Y_acc[75:100],Z_acc[75:100],X_gyro[75:100],Y_gyro[75:100],Z_gyro[75:100],
                                  X_acc[100:125],Y_acc[100:125],Z_acc[100:125],X_gyro[100:125],Y_gyro[100:125],Z_gyro[100:125],
                                  X_acc[125:150],Y_acc[125:150],Z_acc[125:150],X_gyro[125:150],Y_gyro[125:150],Z_gyro[125:150]])

                
                window2=np.array([[X_acc[0:25],X_acc[25:50],X_acc[50:75],X_acc[75:100],X_acc[100:125],X_acc[125:150]],
                    [Y_acc[0:25],Y_acc[25:50],Y_acc[50:75],Y_acc[75:100],Y_acc[100:125],Y_acc[125:150]],
                    [Z_acc[0:25],Z_acc[25:50],Z_acc[50:75],Z_acc[75:100],Z_acc[100:125],Z_acc[125:150]]])
               
            
            
                """
                window4=np.array([[X_gyro[0:25],X_gyro[25:50],X_gyro[50:75],X_gyro[75:100],X_gyro[100:125],X_gyro[125:150]],
                    [Y_gyro[0:25],Y_gyro[25:50],Y_gyro[50:75],Y_gyro[75:100],Y_gyro[100:125],Y_gyro[125:150]],
                    [Z_gyro[0:25],Z_gyro[25:50],Z_gyro[50:75],Z_gyro[75:100],Z_gyro[100:125],Z_gyro[125:150]]])
                """
                
                
                
                window_mean=window.mean(axis=-1)
                window_stddev=window.std(axis=-1)
                window_median=np.median(window,axis=-1)
                window_percent25=np.percentile(window,25,axis=-1)
                window_percent75=np.percentile(window,75,axis=-1)
                window_feature=np.array([[window_mean],[window_stddev],[window_median],[window_percent25],[window_percent75]])
                window_feature=window_feature.reshape(180)
                
                #Drop outliers1 
                xm=window_stddev[[0,6,12,18,24,30]].mean(axis=0)
                ym=window_stddev[[1,7,13,19,25,31]].mean(axis=0)
                zm=window_stddev[[2,8,14,20,26,32]].mean(axis=0)
                action=np.mean([xm,ym,zm])
                if(action<0.65):
                    label_count[label]-=1
                    continue
                
                window2=window2.reshape(-1)
                #window_RMS=np.roots(np.square(window2[0:6]).mean(axis=-1))
                #if (label_count[label]>=0):
                data_set.append(window_feature)
                data_set2.append(window2)
                data_label.append(label)
            
                #else:
                #    test_set.append(window_feature)
                #    test_label.append(label)
                #    testlabel_count[label]+=1
                
            from sklearn.manifold import TSNE
            from sklearn.decomposition import PCA
            from sklearn.decomposition import MiniBatchDictionaryLearning
            from sklearn.decomposition import IncrementalPCA
            from sklearn.decomposition import TruncatedSVD


            print(np.shape(data_set2))

            data_set2=np.array(data_set2)
            X_embedded2 = MiniBatchDictionaryLearning(n_components=3,n_iter=1000).fit_transform(data_set2)
            #fig = plt.figure()
            #ax2 = fig.add_subplot(111, projection='3d')
            #for x in X_embedded2:
            #    ax2.scatter(x[0],x[1],x[2]) 
            if(int(label)==12):
                from scipy import stats
                z = np.abs(stats.zscore(X_embedded2))
                #X_embedded2 = X_embedded2[(abs(z) <1.8).all(axis=1)]
                data_set_arr=np.array(data_set)
                data_label_arr=np.array(data_label)
                data_set_arr=data_set_arr[(abs(z) <1.5).all(axis=1)]
                data_label_arr=data_label_arr[(abs(z) <1.5).all(axis=1)]
            else:
                if(int(label)==10):
                    z = np.abs(stats.zscore(X_embedded2))
                    #X_embedded2 = X_embedded2[(abs(z) <1.8).all(axis=1)]
                    data_set_arr=np.array(data_set)
                    data_label_arr=np.array(data_label)
                    data_set_arr=data_set_arr[(abs(z) <1.8).all(axis=1)]
                    data_label_arr=data_label_arr[(abs(z) <1.8).all(axis=1)]
                
                else:
                    z = np.abs(stats.zscore(X_embedded2))
                    #X_embedded2 = X_embedded2[(abs(z) <1.8).all(axis=1)]
                    data_set_arr=np.array(data_set)
                    data_label_arr=np.array(data_label)
                    data_set_arr=data_set_arr[(abs(z) <2).all(axis=1)]
                    data_label_arr=data_label_arr[(abs(z) <2).all(axis=1)]

            #print(np.shape(data_set_arr))
            #print(np.shape(data_label_arr))

            for x in data_set_arr:
                training_set.append(x)
            for y in data_label_arr:
                training_label.append(y)
                
                

            #fig2 = plt.figure()
            #ax2 = fig2.add_subplot(111, projection='3d')
            #for x in X_embedded2:
            #    ax2.scatter(x[0],x[1],x[2]) 


label :  12
(248, 450)
label :  8
(68, 450)
label :  7
(68, 450)
label :  5
(136, 450)
label :  13
(36, 450)
label :  9
(72, 450)
label :  10
(136, 450)
label :  0
(124, 450)
label :  1
(130, 450)
label :  6
(228, 450)


In [95]:
X=training_set
y=training_label

print(np.shape(X))

(1067, 180)


In [96]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

#X=preprocessing.minmax_scale(training_set)
#X=preprocessing.robust_scale(training_set)
#X=preprocessing.quantile_transform(training_set)

X=training_set
y=training_label
RF=RandomForestClassifier()
param_grid={
 'n_estimators':[50,80,100,120],
    'max_features':['auto','log2','sqrt'],
    'max_depth':[None]
    #'criterion':['gini','entropy']
}
grid=GridSearchCV(estimator=RF,param_grid=param_grid,cv=10,scoring='accuracy')

In [97]:
grid.fit(X,y)

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             ii

In [98]:
grid.cv_results_

{'mean_fit_time': array([0.24508076, 0.38825901, 0.49007437, 0.70930445, 0.17449555,
        0.24308171, 0.32847958, 0.36074026, 0.24198623, 0.41491141,
        0.58475325, 0.63078558]),
 'std_fit_time': array([0.00484205, 0.00376243, 0.00379696, 0.08448749, 0.0309737 ,
        0.00804798, 0.04614659, 0.00694797, 0.00136083, 0.0383212 ,
        0.05877216, 0.05930832]),
 'mean_score_time': array([0.003426  , 0.00507567, 0.00620017, 0.00929348, 0.00402853,
        0.00545461, 0.00677326, 0.00749881, 0.00338416, 0.00617349,
        0.00820725, 0.00885468]),
 'std_score_time': array([9.61850434e-05, 3.31430039e-05, 6.24402009e-05, 4.25194366e-03,
        9.53696390e-04, 5.92733983e-04, 1.26956908e-03, 3.10717536e-04,
        3.67035781e-05, 2.83045976e-03, 3.11443557e-03, 3.16664944e-03]),
 'param_max_depth': masked_array(data=[None, None, None, None, None, None, None, None, None,
                    None, None, None],
              mask=[False, False, False, False, False, False, False, F

In [99]:
grid.best_params_

{'max_depth': None, 'max_features': 'auto', 'n_estimators': 120}

In [100]:
RF=RandomForestClassifier(n_estimators=120,max_features='auto',max_depth=None)
RF.fit(X,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=120,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [101]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
X=preprocessing.quantile_transform(training_set)
ETC=ExtraTreesClassifier()

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/preprocessing/data.py:2553: FutureWarning: The default value of `copy` will change from False to True in 0.23 in order to make it more consistent with the default `copy` values of other functions in :mod:`sklearn.preprocessing.data` and prevent unexpected side effects by modifying the value of `X` inplace. To avoid inplace modifications of `X`, it is recommended to explicitly set `copy=True`
  FutureWarning)


In [102]:
param_grid={
 'n_estimators':[100,200,300],
    'max_features':['auto','sqrt','log2'],
    'max_depth':[17,20,25,30]
    #'criterion':['gini','entropy']
}

grid=GridSearchCV(estimator=ETC,param_grid=param_grid,cv=10,scoring='accuracy')

In [103]:
grid.fit(X,y)


/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=ExtraTreesClassifier(bootstrap=False, class_weight=None,
                                            criterion='gini', max_depth=None,
                                            max_features='auto',
                                            max_leaf_nodes=None,
                                            min_impurity_decrease=0.0,
                                            min_impurity_split=None,
                                            min_samples_leaf=1,
                                            min_samples_split=2,
                                            min_weight_fraction_leaf=0.0,
                                            n_estimators='warn', n_jobs=None,
                                            oob_score=False, random_state=None,
                                            verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': [17, 20,

In [104]:
grid.cv_results_

{'mean_fit_time': array([0.13440371, 0.31285665, 0.38421807, 0.12859495, 0.25634625,
        0.38297377, 0.09940097, 0.1976511 , 0.29724841, 0.12877414,
        0.2565161 , 0.38348498, 0.13316631, 0.26105771, 0.38990538,
        0.10086179, 0.20188918, 0.30208225, 0.16014745, 0.30043721,
        0.40637705, 0.132617  , 0.26469038, 0.4522409 , 0.12255132,
        0.30296898, 0.31535351, 0.14095981, 0.33068111, 0.44781864,
        0.16498342, 0.3009747 , 0.4140178 , 0.10473809, 0.20678358,
        0.33048036]),
 'std_fit_time': array([0.01183615, 0.07539146, 0.00241232, 0.00061637, 0.00282261,
        0.00196359, 0.00089811, 0.00174214, 0.00380217, 0.00108264,
        0.00108646, 0.00116009, 0.00400281, 0.0035212 , 0.00161117,
        0.00039402, 0.0015638 , 0.00340236, 0.0427713 , 0.05423099,
        0.02211028, 0.00068231, 0.00091348, 0.09792609, 0.01222017,
        0.10312692, 0.00683733, 0.01219212, 0.0414157 , 0.04442885,
        0.0346028 , 0.04107012, 0.02646179, 0.00070413, 0.001

In [105]:
grid.best_params_

{'max_depth': 17, 'max_features': 'log2', 'n_estimators': 300}

In [106]:
ETC=ExtraTreesClassifier(max_depth=17,max_features='log2',n_estimators=300).fit(X,y)

In [107]:
from sklearn.model_selection import cross_val_predict

y_pred = cross_val_predict(ETC, X, y, cv=10)
conf_mat = confusion_matrix(y, y_pred)
print(conf_mat)

[[109   0   0   4   0   0   0   0   3   0]
 [  0 115   4   3   0   0   0   0   0   0]
 [  0   0 116   8   0   0   0   0   0   0]
 [  0   0   0 204   0   0   0   0   0   0]
 [  0   0   0   0  52   0   0   0   4   0]
 [  0   4   0   5   0  48   0   0   0   0]
 [  0   0   0   0   0   0  48  12   4   0]
 [  0   0   0   0   0   0   1 110   9   0]
 [  0   0   0   0   0   0   4  16 152   4]
 [  0   0   0   0   0   0   0   0   0  28]]


In [14]:
class_names=['face','pouring','clinking','drinking1','tissue','drink2','spoon','chopstick','fork','stir']

X=training_set
y=training_label

#print((test_pred_proba))
#for row in test_pred_proba:
#    print(np.max([row]))
test_pred3=ETC.predict(preprocessing.quantile_transform(X))
print(accuracy_score(test_pred3,training_label))
print(confusion_matrix(test_pred3, training_label))

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/preprocessing/data.py:2553: FutureWarning: The default value of `copy` will change from False to True in 0.23 in order to make it more consistent with the default `copy` values of other functions in :mod:`sklearn.preprocessing.data` and prevent unexpected side effects by modifying the value of `X` inplace. To avoid inplace modifications of `X`, it is recommended to explicitly set `copy=True`
  FutureWarning)


1.0
[[116   0   0   0   0   0   0   0   0   0]
 [  0   4   0   0   0   0   0   0   0   0]
 [  0   0 120   0   0   0   0   0   0   0]
 [  0   0   0 200   0   0   0   0   0   0]
 [  0   0   0   0  57   0   0   0   0   0]
 [  0   0   0   0   0  56   0   0   0   0]
 [  0   0   0   0   0   0  68   0   0   0]
 [  0   0   0   0   0   0   0 128   0   0]
 [  0   0   0   0   0   0   0   0 224   0]
 [  0   0   0   0   0   0   0   0   0  28]]


In [108]:
#SVM
#prepare the data
#prepare the model
#X=preprocessing.minmax_scale(training_set)
#X=preprocessing.robust_scale(training_set)
X=preprocessing.quantile_transform(training_set)
y=training_label
clf = sklearn.svm.SVC(gamma='scale',tol=0.1,probability=True)
scores=cross_val_score(clf,X,y,cv=10,scoring='accuracy')
print(scores.mean())

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/preprocessing/data.py:2553: FutureWarning: The default value of `copy` will change from False to True in 0.23 in order to make it more consistent with the default `copy` values of other functions in :mod:`sklearn.preprocessing.data` and prevent unexpected side effects by modifying the value of `X` inplace. To avoid inplace modifications of `X`, it is recommended to explicitly set `copy=True`
  FutureWarning)


0.9030282821749248


In [109]:
param_grid={
 #'kernel':['linear','poly','rbf','sigmoid'], rbf to be best acc
    'C':[10,20,50,100]}

grid=GridSearchCV(estimator=clf,param_grid=param_grid,cv=10,scoring='accuracy')

In [110]:
grid.fit(X,y)
grid.cv_results_

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'mean_fit_time': array([0.53390267, 0.54411323, 0.53812099, 0.51325831]),
 'std_fit_time': array([0.03704927, 0.06052332, 0.04098229, 0.02844576]),
 'mean_score_time': array([0.01463621, 0.01661682, 0.01529706, 0.01485248]),
 'std_score_time': array([0.00166451, 0.00511245, 0.0024762 , 0.00188217]),
 'param_C': masked_array(data=[10, 20, 50, 100],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 10}, {'C': 20}, {'C': 50}, {'C': 100}],
 'split0_test_score': array([0.90990991, 0.90990991, 0.90990991, 0.90990991]),
 'split1_test_score': array([0.95495495, 0.95495495, 0.95495495, 0.95495495]),
 'split2_test_score': array([0.86363636, 0.86363636, 0.86363636, 0.86363636]),
 'split3_test_score': array([0.95454545, 0.95454545, 0.95454545, 0.95454545]),
 'split4_test_score': array([0.96261682, 0.96261682, 0.96261682, 0.96261682]),
 'split5_test_score': array([0.99065421, 0.99065421, 0.99065421, 0.99065421]),
 'split6_test_sc

In [111]:
grid.best_params_

{'C': 10}

In [112]:
SVC=sklearn.svm.SVC(10,probability=True).fit(X,y)

/home/parkj0/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [113]:
#save the model

from joblib import dump, load
filename='./svm.sav'
pickle.dump(SVC,open(filename,'wb'))

In [114]:
filename='./rf.sav'
pickle.dump(RF,open(filename,'wb'))
dump(RF,'rf.joblib')

['rf.joblib']

In [115]:
filename='./ETC.sav'
pickle.dump(ETC,open(filename,'wb'))
dump(ETC,'ETC.joblib')

['ETC.joblib']